# Exercícios de Big Data com Spark + Iceberg + Hive Metastore + HDFS

Este notebook contém as soluções para os 20 exercícios de Big Data usando:
- **Apache Spark** para processamento distribuído
- **Apache Iceberg** para formato de tabela
- **Hive Metastore** para metadados
- **HDFS** para armazenamento distribuído

**Pré-requisitos:**
- Ambiente `lab` configurado com todos os serviços
- Jupyter Notebook rodando dentro do container
- Conectividade com HDFS, Hive Metastore e catálogo Iceberg

## 1. Setup e Configuração

Primeiro, vamos configurar o Spark com o catálogo Iceberg e verificar a conectividade.

In [ ]:
# Importar bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import *

# Criar sessão Spark com configuração Iceberg
spark = SparkSession.builder \
    .appName("BigDataExercicios") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.spark_catalog.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.iceberg.warehouse", "hdfs://namenode:9000/warehouse") \
    .getOrCreate()

# Configurar nível de log
spark.sparkContext.setLogLevel("WARN")

print("✅ Spark Session criada com sucesso!")
print(f"Spark Version: {spark.version}")
print(f"Default Catalog: {spark.conf.get('spark.sql.defaultCatalog', 'spark_catalog')}")

In [ ]:
# Verificar conectividade com HDFS
try:
    spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem.get(
        spark.sparkContext._jvm.java.net.URI("hdfs://namenode:9000"),
        spark.sparkContext._jsc.hadoopConfiguration()
    ).listStatus(spark.sparkContext._jvm.org.apache.hadoop.fs.Path("/"))
    print("✅ HDFS acessível!")
except Exception as e:
    print(f"❌ Erro ao acessar HDFS: {e}")

# Verificar catálogos disponíveis
print("\n📋 Catálogos disponíveis:")
spark.sql("SHOW CATALOGS").show()

## 2. Exercício 1: Criar um DataFrame simples

In [ ]:
# Exercício 1: Criar um DataFrame com três linhas e duas colunas (id, nome)
data = [(1, "João"), (2, "Maria"), (3, "Pedro")]
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("nome", StringType(), True)
])

df_simples = spark.createDataFrame(data, schema)

print("📊 DataFrame criado:")
df_simples.show()

print("📋 Schema do DataFrame:")
df_simples.printSchema()

print(f"📈 Número de registros: {df_simples.count()}")

## 3. Exercício 2: Salvar DataFrame no HDFS como CSV

In [ ]:
# Exercício 2: Salvar DataFrame no HDFS como CSV
hdfs_path = "hdfs://namenode:9000/data/ex1.csv"

# Criar um DataFrame para salvar
data_ex2 = [(1, "Alice"), (2, "Bob"), (3, "Charlie"), (4, "Diana")]
df_ex2 = spark.createDataFrame(data_ex2, schema)

print("📊 DataFrame a ser salvo:")
df_ex2.show()

# Salvar como CSV no HDFS (mode='overwrite' para sobrescrever se existir)
df_ex2.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(hdfs_path)

print(f"✅ DataFrame salvo em: {hdfs_path}")

# Verificar se o arquivo foi criado
try:
    # Listar arquivos no diretório
    import subprocess
    result = subprocess.run(['hdfs', 'dfs', '-ls', '/data/'], 
                          capture_output=True, text=True)
    print("📁 Arquivos no diretório /data/:")
    print(result.stdout)
except Exception as e:
    print(f"⚠️ Não foi possível listar arquivos: {e}")

## 4. Exercício 3: Ler CSV do HDFS

In [ ]:
# Exercício 3: Ler CSV do HDFS
df_lido = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(hdfs_path)

print("📖 DataFrame lido do HDFS:")
df_lido.show()

print("📋 Schema inferido:")
df_lido.printSchema()

print(f"📈 Número de registros lidos: {df_lido.count()}")

# Verificar se os dados são iguais
print("\n🔍 Comparação com DataFrame original:")
print("DataFrame original:")
df_ex2.show()
print("DataFrame lido:")
df_lido.show()

## 5. Exercício 4: Criar namespace Iceberg

In [ ]:
# Exercício 4: Criar namespace Iceberg
print("🗂️ Criando namespace 'lab.db'...")

# Criar namespace
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.lab")
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.lab.db")

print("✅ Namespace 'lab.db' criado com sucesso!")

# Verificar namespaces criados
print("\n📋 Namespaces disponíveis no catálogo iceberg:")
try:
    spark.sql("SHOW NAMESPACES IN iceberg").show()
except Exception as e:
    print(f"⚠️ Erro ao mostrar namespaces: {e}")
    # Alternativa
    spark.sql("SHOW DATABASES").show()

## 6. Exercício 5: Criar tabela Iceberg

In [ ]:
# Exercício 5: Criar tabela Iceberg pessoas
print("🗃️ Criando tabela Iceberg 'pessoas'...")

create_table_sql = """
CREATE TABLE IF NOT EXISTS iceberg.lab.db.pessoas (
    id INT,
    nome STRING
) USING ICEBERG
"""

spark.sql(create_table_sql)
print("✅ Tabela 'lab.db.pessoas' criada com sucesso!")

# Verificar se a tabela foi criada
print("\n📋 Tabelas no namespace lab.db:")
try:
    spark.sql("SHOW TABLES IN iceberg.lab.db").show()
except Exception as e:
    print(f"⚠️ Erro: {e}")
    # Alternativa
    spark.sql("SHOW TABLES").show()

# Mostrar estrutura da tabela
print("\n📊 Estrutura da tabela pessoas:")
spark.sql("DESCRIBE iceberg.lab.db.pessoas").show()

## 7. Exercício 6: Inserir dados na tabela Iceberg

In [ ]:
# Exercício 6: Inserir dados na tabela Iceberg
print("📝 Inserindo dados na tabela pessoas...")

# Inserir 3 registros
insert_sql = """
INSERT INTO iceberg.lab.db.pessoas VALUES 
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie')
"""

spark.sql(insert_sql)
print("✅ Dados inseridos com sucesso!")

# Verificar inserção
print("\n📊 Dados na tabela pessoas:")
spark.sql("SELECT * FROM iceberg.lab.db.pessoas").show()

## 8. Exercício 7: Ler tabela Iceberg

In [ ]:
# Exercício 7: Ler tabela Iceberg
print("📖 Lendo dados da tabela pessoas...")

# Query SELECT simples
df_pessoas = spark.sql("SELECT * FROM iceberg.lab.db.pessoas")
df_pessoas.show()

# Também podemos usar a API DataFrame
df_pessoas_api = spark.table("iceberg.lab.db.pessoas")
print("\n📊 Usando API DataFrame:")
df_pessoas_api.show()

# Mostrar informações do DataFrame
print(f"📈 Total de registros: {df_pessoas.count()}")
print("📋 Schema:")
df_pessoas.printSchema()

## 9. Exercício 8: Contar registros

In [ ]:
# Exercício 8: Contar registros
print("🔢 Contando registros na tabela pessoas...")

# Usando SQL COUNT
count_result = spark.sql("SELECT COUNT(*) as total FROM iceberg.lab.db.pessoas")
count_result.show()

# Capturar o valor para uso
total_registros = count_result.collect()[0]['total']
print(f"📊 Total de registros na tabela pessoas: {total_registros}")

# Alternativa usando DataFrame API
total_api = spark.table("iceberg.lab.db.pessoas").count()
print(f"📊 Total usando DataFrame API: {total_api}")

## 10. Exercício 9: Atualizar um registro

In [ ]:
# Exercício 9: Atualizar um registro
print("✏️ Atualizando registro na tabela pessoas...")

# Mostrar dados antes da atualização
print("📊 Dados ANTES da atualização:")
spark.sql("SELECT * FROM iceberg.lab.db.pessoas").show()

# Atualizar Alice para Alice Silva
update_sql = """
UPDATE iceberg.lab.db.pessoas 
SET nome = 'Alice Silva' 
WHERE nome = 'Alice'
"""

spark.sql(update_sql)
print("✅ Registro atualizado com sucesso!")

# Mostrar dados após a atualização
print("📊 Dados APÓS a atualização:")
spark.sql("SELECT * FROM iceberg.lab.db.pessoas").show()

## 11. Exercício 10: Deletar um registro

In [ ]:
# Exercício 10: Deletar um registro
print("🗑️ Deletando registro da tabela pessoas...")

# Mostrar dados antes da exclusão
print("📊 Dados ANTES da exclusão:")
spark.sql("SELECT * FROM iceberg.lab.db.pessoas").show()

# Deletar Bob
delete_sql = """
DELETE FROM iceberg.lab.db.pessoas 
WHERE nome = 'Bob'
"""

spark.sql(delete_sql)
print("✅ Registro deletado com sucesso!")

# Mostrar dados após a exclusão
print("📊 Dados APÓS a exclusão:")
spark.sql("SELECT * FROM iceberg.lab.db.pessoas").show()

print(f"📈 Total de registros após exclusão: {spark.sql('SELECT COUNT(*) FROM iceberg.lab.db.pessoas').collect()[0][0]}")

## 12. Exercício 11: Criar tabela particionada

In [ ]:
# Exercício 11: Criar tabela particionada por ano
print("🗂️ Criando tabela particionada 'vendas'...")

create_vendas_sql = """
CREATE TABLE IF NOT EXISTS iceberg.lab.db.vendas (
    id INT,
    valor DOUBLE,
    ano INT
) USING ICEBERG
PARTITIONED BY (ano)
"""

spark.sql(create_vendas_sql)
print("✅ Tabela 'vendas' particionada criada com sucesso!")

# Verificar estrutura da tabela
print("\n📊 Estrutura da tabela vendas:")
spark.sql("DESCRIBE iceberg.lab.db.vendas").show()

# Mostrar informações de particionamento
print("\n🗂️ Informações de particionamento:")
spark.sql("SHOW PARTITIONS iceberg.lab.db.vendas").show()

## 13. Exercício 12: Inserir dados particionados

In [ ]:
# Exercício 12: Inserir dados particionados
print("📝 Inserindo dados particionados na tabela vendas...")

insert_vendas_sql = """
INSERT INTO iceberg.lab.db.vendas VALUES 
    (1, 1000.50, 2022),
    (2, 1500.75, 2022),
    (3, 2000.00, 2023),
    (4, 2500.25, 2023),
    (5, 3000.00, 2023),
    (6, 1750.80, 2024),
    (7, 2200.60, 2024)
"""

spark.sql(insert_vendas_sql)
print("✅ Dados particionados inseridos com sucesso!")

# Verificar dados inseridos
print("\n📊 Dados na tabela vendas:")
spark.sql("SELECT * FROM iceberg.lab.db.vendas ORDER BY ano, id").show()

# Verificar partições criadas
print("\n🗂️ Partições criadas:")
try:
    spark.sql("SHOW PARTITIONS iceberg.lab.db.vendas").show()
except:
    print("Informação de partições não disponível via SHOW PARTITIONS")

print(f"\n📈 Total de registros: {spark.sql('SELECT COUNT(*) FROM iceberg.lab.db.vendas').collect()[0][0]}")

## 14. Exercício 13: Consultar apenas um particionamento

In [ ]:
# Exercício 13: Consultar apenas vendas de 2023
print("🔍 Consultando apenas vendas do ano 2023...")

vendas_2023 = spark.sql("SELECT * FROM iceberg.lab.db.vendas WHERE ano = 2023")
vendas_2023.show()

print(f"📊 Total de vendas em 2023: {vendas_2023.count()}")
print(f"💰 Valor total vendido em 2023: {vendas_2023.agg({'valor': 'sum'}).collect()[0][0]}")

# Comparar com outros anos
print("\n📈 Resumo por ano:")
spark.sql("""
    SELECT ano, 
           COUNT(*) as qtd_vendas, 
           SUM(valor) as total_valor,
           AVG(valor) as valor_medio
    FROM iceberg.lab.db.vendas 
    GROUP BY ano 
    ORDER BY ano
""").show()

## 15. Exercício 14: Ver metadados da tabela

In [ ]:
# Exercício 14: Ver metadados da tabela
print("📋 Visualizando metadados da tabela vendas...")

# DESCRIBE HISTORY - histórico da tabela
print("📜 Histórico da tabela (DESCRIBE HISTORY):")
try:
    spark.sql("DESCRIBE HISTORY iceberg.lab.db.vendas").show(truncate=False)
except Exception as e:
    print(f"⚠️ Erro ao mostrar histórico: {e}")

print("\n" + "="*80)

# DESCRIBE DETAIL - detalhes da tabela  
print("🔍 Detalhes da tabela (DESCRIBE DETAIL):")
try:
    spark.sql("DESCRIBE DETAIL iceberg.lab.db.vendas").show(truncate=False)
except Exception as e:
    print(f"⚠️ Erro ao mostrar detalhes: {e}")

print("\n" + "="*80)

# Informações adicionais
print("ℹ️ Informações adicionais:")
print("📊 Estrutura da tabela:")
spark.sql("DESCRIBE iceberg.lab.db.vendas").show()

print("📁 Propriedades da tabela:")
try:
    spark.sql("SHOW TBLPROPERTIES iceberg.lab.db.vendas").show()
except Exception as e:
    print(f"⚠️ Propriedades não disponíveis: {e}")

## 16. Exercício 15: Criar tabela Iceberg a partir de DataFrame

In [ ]:
# Exercício 15: Criar tabela Iceberg a partir de DataFrame
print("🔄 Criando tabela Iceberg diretamente a partir de DataFrame...")

# Criar DataFrame artificial com dados de produtos
data_produtos = [
    (1, "Notebook", 2500.00, "Eletrônicos"),
    (2, "Mouse", 50.00, "Eletrônicos"),
    (3, "Teclado", 150.00, "Eletrônicos"),
    (4, "Monitor", 800.00, "Eletrônicos"),
    (5, "Cadeira", 400.00, "Móveis"),
    (6, "Mesa", 600.00, "Móveis")
]

schema_produtos = StructType([
    StructField("id", IntegerType(), True),
    StructField("nome", StringType(), True),
    StructField("preco", DoubleType(), True),
    StructField("categoria", StringType(), True)
])

df_produtos = spark.createDataFrame(data_produtos, schema_produtos)

print("📊 DataFrame produtos criado:")
df_produtos.show()

# Criar tabela Iceberg usando writeTo()
df_produtos.writeTo("iceberg.lab.db.tabela_df").createOrReplace()
print("✅ Tabela 'tabela_df' criada com sucesso a partir do DataFrame!")

# Verificar tabela criada
print("\n📋 Dados na nova tabela:")
spark.sql("SELECT * FROM iceberg.lab.db.tabela_df").show()

print("\n📊 Estatísticas por categoria:")
spark.sql("""
    SELECT categoria, 
           COUNT(*) as qtd_produtos,
           AVG(preco) as preco_medio,
           MAX(preco) as preco_maximo
    FROM iceberg.lab.db.tabela_df 
    GROUP BY categoria
""").show()

## 17. Exercício 16: Converter tabela para Iceberg

In [ ]:
# Exercício 16: Converter tabela Parquet para Iceberg
print("🔄 Criando tabela Parquet e convertendo para Iceberg...")

# Primeiro, criar uma tabela Parquet simples
data_temp = [
    (1, "Dados temporários 1"),
    (2, "Dados temporários 2"),
    (3, "Dados temporários 3")
]

df_temp = spark.createDataFrame(data_temp, StructType([
    StructField("id", IntegerType(), True),
    StructField("descricao", StringType(), True)
]))

# Salvar como tabela Parquet
df_temp.write.mode("overwrite").saveAsTable("temp_parquet")
print("✅ Tabela Parquet 'temp_parquet' criada!")

# Verificar tabela criada
print("\n📊 Dados na tabela Parquet:")
spark.sql("SELECT * FROM temp_parquet").show()

# Converter para Iceberg (simulação - o comando exato pode variar)
print("\n🔄 Convertendo para formato Iceberg...")
try:
    # Método 1: Recriar como Iceberg
    df_temp.writeTo("iceberg.lab.db.temp_iceberg").createOrReplace()
    print("✅ Tabela convertida para Iceberg como 'temp_iceberg'!")
    
    # Verificar conversão
    print("\n📊 Dados na tabela Iceberg convertida:")
    spark.sql("SELECT * FROM iceberg.lab.db.temp_iceberg").show()
    
except Exception as e:
    print(f"⚠️ Erro na conversão: {e}")
    print("💡 Nota: A conversão direta pode requerer configurações específicas do ambiente")

## 18. Exercício 17: Leitura incremental (Time Travel)

In [ ]:
# Exercício 17: Time Travel - Leitura incremental
print("⏰ Demonstrando funcionalidade Time Travel do Iceberg...")

# Primeiro, vamos ver o histórico atual da tabela vendas
print("📜 Histórico atual da tabela vendas:")
try:
    history_df = spark.sql("DESCRIBE HISTORY iceberg.lab.db.vendas")
    history_df.show(truncate=False)
    
    # Capturar snapshots disponíveis
    snapshots = history_df.collect()
    if len(snapshots) > 0:
        print(f"\n📊 Total de snapshots disponíveis: {len(snapshots)}")
        
        # Tentar acessar versão anterior (snapshot 1)
        print("\n⏮️ Acessando dados da versão 1:")
        try:
            spark.sql("SELECT * FROM iceberg.lab.db.vendas VERSION AS OF 1").show()
        except Exception as e:
            print(f"⚠️ Não foi possível acessar versão 1: {e}")
            
        # Mostrar dados da versão atual
        print("\n📊 Dados da versão atual:")
        spark.sql("SELECT * FROM iceberg.lab.db.vendas").show()
        
        # Exemplo de time travel por timestamp (se disponível)
        print("\n⏰ Tentando time travel por timestamp...")
        try:
            # Pegar timestamp do primeiro snapshot
            first_snapshot = snapshots[-1]  # Mais antigo
            timestamp = first_snapshot['made_current_at']
            print(f"Tentando acessar dados do timestamp: {timestamp}")
            spark.sql(f"SELECT * FROM iceberg.lab.db.vendas TIMESTAMP AS OF '{timestamp}'").show()
        except Exception as e:
            print(f"⚠️ Time travel por timestamp não disponível: {e}")
    else:
        print("⚠️ Nenhum snapshot encontrado")
        
except Exception as e:
    print(f"⚠️ Erro ao acessar histórico: {e}")
    print("💡 Time Travel requer que existam múltiplas versões da tabela")

## 19. Exercício 18: Exportar tabela Iceberg para CSV

In [ ]:
# Exercício 18: Exportar tabela Iceberg para CSV
print("📤 Exportando tabela Iceberg para CSV no HDFS...")

# Ler dados da tabela Iceberg
df_vendas_export = spark.sql("SELECT * FROM iceberg.lab.db.vendas")

print("📊 Dados a serem exportados:")
df_vendas_export.show()

# Caminho de destino no HDFS
export_path = "hdfs://namenode:9000/export/vendas.csv"

# Exportar para CSV
df_vendas_export.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(export_path)

print(f"✅ Dados exportados com sucesso para: {export_path}")

# Verificar se o arquivo foi criado
try:
    import subprocess
    result = subprocess.run(['hdfs', 'dfs', '-ls', '/export/'], 
                          capture_output=True, text=True)
    print("\n📁 Arquivos no diretório /export/:")
    print(result.stdout)
except Exception as e:
    print(f"⚠️ Não foi possível listar arquivos: {e}")

# Verificar conteúdo do arquivo exportado
print("\n🔍 Verificando dados exportados:")
df_verificacao = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(export_path)

df_verificacao.show()
print(f"📈 Total de registros exportados: {df_verificacao.count()}")

## 20. Exercício 19: Configuração para Dashboard no Superset

In [ ]:
# Exercício 19: Preparação para Dashboard no Superset
print("📊 Preparando dados para Dashboard no Superset...")

# Criar uma view agregada para facilitar visualizações
print("📈 Criando view agregada para dashboards...")

create_view_sql = """
CREATE OR REPLACE VIEW iceberg.lab.db.vendas_dashboard AS
SELECT 
    ano,
    COUNT(*) as total_vendas,
    SUM(valor) as receita_total,
    AVG(valor) as ticket_medio,
    MIN(valor) as menor_venda,
    MAX(valor) as maior_venda
FROM iceberg.lab.db.vendas
GROUP BY ano
"""

spark.sql(create_view_sql)
print("✅ View 'vendas_dashboard' criada!")

# Verificar dados da view
print("\n📊 Dados da view para dashboard:")
spark.sql("SELECT * FROM iceberg.lab.db.vendas_dashboard ORDER BY ano").show()

# Criar dados de exemplo adicionais para visualizações mais ricas
print("\n📝 Criando tabela de vendas detalhadas para dashboards...")

vendas_detalhadas_sql = """
CREATE OR REPLACE TABLE iceberg.lab.db.vendas_detalhadas
USING ICEBERG AS
SELECT 
    id,
    valor,
    ano,
    CASE 
        WHEN valor < 1000 THEN 'Baixo'
        WHEN valor < 2000 THEN 'Médio' 
        ELSE 'Alto'
    END as categoria_valor,
    CASE 
        WHEN id % 4 = 0 THEN 'Q1'
        WHEN id % 4 = 1 THEN 'Q2'
        WHEN id % 4 = 2 THEN 'Q3'
        ELSE 'Q4'
    END as trimestre,
    valor * 0.1 as comissao
FROM iceberg.lab.db.vendas
"""

spark.sql(vendas_detalhadas_sql)
print("✅ Tabela 'vendas_detalhadas' criada!")

print("\n📊 Preview dos dados detalhados:")
spark.sql("SELECT * FROM iceberg.lab.db.vendas_detalhadas ORDER BY ano, id").show()

print("""
📋 INSTRUÇÕES PARA SUPERSET:

1. Conectar ao Trino:
   - Host: trino (ou IP do container Trino)
   - Porta: 8080
   - Usuário: admin
   - Catálogo: iceberg
   - Schema: lab.db

2. URL de conexão no Superset:
   trino://admin@trino:8080/iceberg/lab.db

3. Tabelas disponíveis para visualização:
   - iceberg.lab.db.vendas (dados brutos)
   - iceberg.lab.db.vendas_dashboard (dados agregados)
   - iceberg.lab.db.vendas_detalhadas (dados com categorias)
   - iceberg.lab.db.pessoas (dados de pessoas)
   - iceberg.lab.db.tabela_df (dados de produtos)

4. Exemplos de visualizações:
   - Gráfico de barras: receita por ano
   - Gráfico de pizza: vendas por categoria
   - Tabela: métricas de vendas por trimestre
""")

## 21. Exercício 20: Consultas via Trino

In [ ]:
# Exercício 20: Simulação de consultas Trino
print("🔍 Simulando consultas que seriam executadas via Trino...")

# Estas são as consultas que você executaria no CLI do Trino ou Superset
trino_queries = [
    "SELECT * FROM iceberg.lab.db.pessoas;",
    "SELECT * FROM iceberg.lab.db.vendas ORDER BY ano, valor DESC;",
    "SELECT ano, SUM(valor) as total FROM iceberg.lab.db.vendas GROUP BY ano;",
    "SELECT categoria, AVG(preco) as preco_medio FROM iceberg.lab.db.tabela_df GROUP BY categoria;"
]

print("📋 Consultas Trino equivalentes:")
for i, query in enumerate(trino_queries, 1):
    print(f"{i}. {query}")

print("\n" + "="*80)
print("🚀 Executando consultas via Spark (equivalente ao Trino):")

# Query 1: Pessoas
print("\n1️⃣ Consulta: Tabela pessoas")
spark.sql("SELECT * FROM iceberg.lab.db.pessoas").show()

# Query 2: Vendas ordenadas
print("\n2️⃣ Consulta: Vendas ordenadas por ano e valor")
spark.sql("SELECT * FROM iceberg.lab.db.vendas ORDER BY ano, valor DESC").show()

# Query 3: Total por ano
print("\n3️⃣ Consulta: Total de vendas por ano")
spark.sql("SELECT ano, SUM(valor) as total FROM iceberg.lab.db.vendas GROUP BY ano ORDER BY ano").show()

# Query 4: Preço médio por categoria
print("\n4️⃣ Consulta: Preço médio por categoria de produtos")
spark.sql("SELECT categoria, AVG(preco) as preco_medio FROM iceberg.lab.db.tabela_df GROUP BY categoria").show()

print("""
💡 COMANDOS TRINO CLI:

Para conectar ao Trino CLI (fora do notebook):
$ trino --server localhost:8080 --catalog iceberg --schema lab.db

Comandos úteis no Trino:
- SHOW CATALOGS;
- SHOW SCHEMAS FROM iceberg;
- SHOW TABLES FROM iceberg.lab.db;
- DESCRIBE iceberg.lab.db.vendas;
""")

## 🎯 Resumo Final e Verificações

In [ ]:
# RESUMO FINAL - Verificação de todos os exercícios
print("🎯 RESUMO FINAL DOS EXERCÍCIOS")
print("="*60)

# Verificar todas as tabelas criadas
print("📋 Tabelas Iceberg criadas:")
try:
    spark.sql("SHOW TABLES IN iceberg.lab.db").show()
except:
    print("⚠️ Erro ao listar tabelas")

print("\n📊 Resumo dos dados:")

# 1. Tabela pessoas
try:
    pessoas_count = spark.sql("SELECT COUNT(*) FROM iceberg.lab.db.pessoas").collect()[0][0]
    print(f"👥 Pessoas: {pessoas_count} registros")
except:
    print("👥 Pessoas: Erro ao contar")

# 2. Tabela vendas
try:
    vendas_count = spark.sql("SELECT COUNT(*) FROM iceberg.lab.db.vendas").collect()[0][0]
    vendas_total = spark.sql("SELECT SUM(valor) FROM iceberg.lab.db.vendas").collect()[0][0]
    print(f"💰 Vendas: {vendas_count} registros, Total: R$ {vendas_total:.2f}")
except:
    print("💰 Vendas: Erro ao contar")

# 3. Tabela produtos
try:
    produtos_count = spark.sql("SELECT COUNT(*) FROM iceberg.lab.db.tabela_df").collect()[0][0]
    print(f"📦 Produtos: {produtos_count} registros")
except:
    print("📦 Produtos: Erro ao contar")

print("\n✅ EXERCÍCIOS COMPLETADOS:")
exercicios = [
    "✓ 1. DataFrame simples criado",
    "✓ 2. DataFrame salvo no HDFS como CSV", 
    "✓ 3. CSV lido do HDFS",
    "✓ 4. Namespace Iceberg criado",
    "✓ 5. Tabela Iceberg criada",
    "✓ 6. Dados inseridos na tabela Iceberg",
    "✓ 7. Tabela Iceberg consultada",
    "✓ 8. Registros contados",
    "✓ 9. Registro atualizado",
    "✓ 10. Registro deletado",
    "✓ 11. Tabela particionada criada",
    "✓ 12. Dados particionados inseridos",
    "✓ 13. Partição específica consultada",
    "✓ 14. Metadados da tabela visualizados",
    "✓ 15. Tabela criada a partir de DataFrame",
    "✓ 16. Conversão para Iceberg demonstrada",
    "✓ 17. Time Travel implementado",
    "✓ 18. Dados exportados para CSV",
    "✓ 19. Preparação para Superset realizada",
    "✓ 20. Consultas Trino simuladas"
]

for exercicio in exercicios:
    print(exercicio)

print(f"\n🎉 PARABÉNS! Todos os {len(exercicios)} exercícios foram completados!")
print("\n💡 PRÓXIMOS PASSOS:")
print("- Conecte o Superset ao Trino para criar dashboards")
print("- Explore funcionalidades avançadas do Iceberg")
print("- Pratique com datasets maiores")
print("- Implemente pipelines de dados completos")

print("\n🚀 Bom estudo com Big Data!")

In [ ]:
# Exercício 10: Deletar um registro
print("🗑️ Deletando registro da tabela pessoas...")

# Mostrar dados antes da deleção
print("📊 Dados ANTES da deleção:")
spark.sql("SELECT * FROM iceberg.lab.db.pessoas").show()

# Deletar Charlie
delete_sql = """
DELETE FROM iceberg.lab.db.pessoas 
WHERE nome = 'Charlie'
"""

spark.sql(delete_sql)
print("✅ Registro deletado com sucesso!")

# Mostrar dados após a deleção
print("📊 Dados APÓS a deleção:")
spark.sql("SELECT * FROM iceberg.lab.db.pessoas").show()

# Contar registros restantes
total_after_delete = spark.sql("SELECT COUNT(*) as total FROM iceberg.lab.db.pessoas").collect()[0]['total']
print(f"📈 Total de registros após deleção: {total_after_delete}")

## 12. Exercício 11: Criar tabela particionada

In [ ]:
# Exercício 11: Criar tabela particionada
print("🗂️ Criando tabela particionada 'vendas'...")

create_partitioned_table_sql = """
CREATE TABLE IF NOT EXISTS iceberg.lab.db.vendas (
    id INT,
    valor DOUBLE,
    ano INT
) USING ICEBERG
PARTITIONED BY (ano)
"""

spark.sql(create_partitioned_table_sql)
print("✅ Tabela particionada 'vendas' criada com sucesso!")

# Verificar estrutura da tabela
print("\n📊 Estrutura da tabela vendas:")
spark.sql("DESCRIBE iceberg.lab.db.vendas").show()

# Mostrar informações de particionamento
print("\n🗂️ Informações de particionamento:")
try:
    spark.sql("DESCRIBE DETAIL iceberg.lab.db.vendas").show()
except Exception as e:
    print(f"⚠️ Não foi possível mostrar detalhes: {e}")

## 13. Exercício 12: Inserir dados particionados

In [ ]:
# Exercício 12: Inserir dados particionados
print("📝 Inserindo dados particionados na tabela vendas...")

# Inserir dados com diferentes anos
insert_vendas_sql = """
INSERT INTO iceberg.lab.db.vendas VALUES 
    (1, 1500.50, 2022),
    (2, 2300.75, 2022),
    (3, 1800.00, 2023),
    (4, 2100.25, 2023),
    (5, 2500.00, 2023),
    (6, 1900.00, 2024),
    (7, 2200.50, 2024)
"""

spark.sql(insert_vendas_sql)
print("✅ Dados inseridos com sucesso!")

# Mostrar todos os dados
print("\n📊 Dados na tabela vendas:")
spark.sql("SELECT * FROM iceberg.lab.db.vendas ORDER BY ano, id").show()

# Mostrar resumo por ano
print("\n📈 Resumo por ano:")
spark.sql("""
    SELECT ano, COUNT(*) as total_vendas, SUM(valor) as total_valor 
    FROM iceberg.lab.db.vendas 
    GROUP BY ano 
    ORDER BY ano
""").show()

## 14. Exercício 13: Consultar apenas um particionamento

In [ ]:
# Exercício 13: Consultar apenas um particionamento
print("🔍 Consultando apenas vendas de 2023...")

# Query para apenas 2023
vendas_2023 = spark.sql("""
    SELECT * FROM iceberg.lab.db.vendas 
    WHERE ano = 2023
    ORDER BY id
""")

vendas_2023.show()

print(f"📊 Total de vendas em 2023: {vendas_2023.count()}")

# Mostrar estatísticas de 2023
print("\n📈 Estatísticas de vendas 2023:")
spark.sql("""
    SELECT 
        COUNT(*) as total_vendas,
        SUM(valor) as valor_total,
        AVG(valor) as valor_medio,
        MIN(valor) as menor_venda,
        MAX(valor) as maior_venda
    FROM iceberg.lab.db.vendas 
    WHERE ano = 2023
""").show()

# Verificar se o Spark está fazendo partition pruning
print("\n🚀 Plano de execução (deve mostrar partition pruning):")
vendas_2023.explain()

## 15. Exercício 14: Ver metadados da tabela

In [ ]:
# Exercício 14: Ver metadados da tabela
print("📋 Verificando metadados da tabela vendas...")

# DESCRIBE HISTORY - mostra histórico de commits/snapshots
print("\n🕒 Histórico da tabela (DESCRIBE HISTORY):")
try:
    spark.sql("DESCRIBE HISTORY iceberg.lab.db.vendas").show(truncate=False)
except Exception as e:
    print(f"⚠️ Erro ao obter histórico: {e}")

# DESCRIBE DETAIL - mostra detalhes da tabela
print("\n🔍 Detalhes da tabela (DESCRIBE DETAIL):")
try:
    spark.sql("DESCRIBE DETAIL iceberg.lab.db.vendas").show(truncate=False)
except Exception as e:
    print(f"⚠️ Erro ao obter detalhes: {e}")

# Informações adicionais do Iceberg
print("\n📊 Estrutura da tabela:")
spark.sql("DESCRIBE iceberg.lab.db.vendas").show()

# Mostrar arquivos da tabela (se disponível)
print("\n📁 Files da tabela:")
try:
    spark.sql("SELECT * FROM iceberg.lab.db.vendas.files").show()
except Exception as e:
    print(f"⚠️ Não foi possível mostrar files: {e}")

## 16. Exercício 15: Criar tabela Iceberg a partir de DataFrame

In [ ]:
# Exercício 15: Criar tabela Iceberg a partir de DataFrame
print("📊 Criando DataFrame artificial e salvando como tabela Iceberg...")

# Criar um DataFrame com dados de produtos
from datetime import datetime
import random

produtos_data = [
    (1, "Notebook", 2500.00, "Eletrônicos", datetime(2024, 1, 15)),
    (2, "Mouse", 45.50, "Eletrônicos", datetime(2024, 1, 16)),
    (3, "Teclado", 120.00, "Eletrônicos", datetime(2024, 1, 17)),
    (4, "Monitor", 800.00, "Eletrônicos", datetime(2024, 1, 18)),
    (5, "Cadeira", 350.00, "Móveis", datetime(2024, 1, 19)),
    (6, "Mesa", 450.00, "Móveis", datetime(2024, 1, 20))
]

produtos_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("nome", StringType(), True),
    StructField("preco", DoubleType(), True),
    StructField("categoria", StringType(), True),
    StructField("data_cadastro", DateType(), True)
])

df_produtos = spark.createDataFrame(produtos_data, produtos_schema)

print("📋 DataFrame criado:")
df_produtos.show()

# Salvar como tabela Iceberg usando writeTo
print("\n💾 Salvando como tabela Iceberg...")
df_produtos.writeTo("iceberg.lab.db.produtos").createOrReplace()

print("✅ Tabela 'produtos' criada com sucesso!")

# Verificar a tabela criada
print("\n📊 Dados da nova tabela:")
spark.sql("SELECT * FROM iceberg.lab.db.produtos").show()

print("\n📋 Schema da tabela:")
spark.sql("DESCRIBE iceberg.lab.db.produtos").show()

## 17. Exercício 16: Converter tabela para Iceberg

In [ ]:
# Exercício 16: Converter tabela para Iceberg
print("🔄 Criando tabela Parquet e convertendo para Iceberg...")

# Primeiro, criar uma tabela Parquet simples
clientes_data = [
    (1, "João Silva", "joao@email.com", "SP"),
    (2, "Maria Santos", "maria@email.com", "RJ"),
    (3, "Pedro Costa", "pedro@email.com", "MG"),
    (4, "Ana Oliveira", "ana@email.com", "SP")
]

clientes_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("nome", StringType(), True),
    StructField("email", StringType(), True),
    StructField("estado", StringType(), True)
])

df_clientes = spark.createDataFrame(clientes_data, clientes_schema)

# Salvar como tabela Parquet no catálogo Hive
print("📁 Criando tabela Parquet...")
df_clientes.write \
    .mode("overwrite") \
    .option("path", "hdfs://namenode:9000/warehouse/clientes_parquet") \
    .saveAsTable("default.clientes_parquet")

print("✅ Tabela Parquet criada!")

# Verificar a tabela Parquet
print("\n📊 Dados da tabela Parquet:")
spark.sql("SELECT * FROM default.clientes_parquet").show()

# Agora converter para Iceberg
print("\n🔄 Convertendo para Iceberg...")
try:
    # Criar tabela Iceberg baseada na Parquet
    spark.sql("""
        CREATE TABLE iceberg.lab.db.clientes
        USING ICEBERG
        AS SELECT * FROM default.clientes_parquet
    """)
    print("✅ Tabela convertida para Iceberg com sucesso!")
    
    # Verificar a tabela Iceberg
    print("\n📊 Dados da tabela Iceberg:")
    spark.sql("SELECT * FROM iceberg.lab.db.clientes").show()
    
except Exception as e:
    print(f"⚠️ Erro na conversão: {e}")
    print("💡 Tentando método alternativo...")
    
    # Método alternativo: ler Parquet e escrever como Iceberg
    df_from_parquet = spark.sql("SELECT * FROM default.clientes_parquet")
    df_from_parquet.writeTo("iceberg.lab.db.clientes").createOrReplace()
    print("✅ Conversão alternativa realizada!")

## 18. Exercício 17: Leitura incremental (Time Travel)

In [ ]:
# Exercício 17: Leitura incremental (Time Travel)
print("⏰ Demonstrando Time Travel com Iceberg...")

# Primeiro, vamos ver o histórico da tabela vendas
print("📋 Histórico atual da tabela vendas:")
try:
    history_df = spark.sql("DESCRIBE HISTORY iceberg.lab.db.vendas")
    history_df.show()
    
    # Pegar o número de snapshots disponíveis
    snapshots = history_df.collect()
    print(f"📊 Total de snapshots disponíveis: {len(snapshots)}")
    
    if len(snapshots) >= 2:
        # Mostrar dados atuais
        print("\n📊 Dados atuais (versão mais recente):")
        spark.sql("SELECT * FROM iceberg.lab.db.vendas ORDER BY id").show()
        
        # Time travel para versão anterior
        print(f"\n⏪ Dados da versão anterior (snapshot {snapshots[1]['snapshot_id']}):")
        spark.sql(f"""
            SELECT * FROM iceberg.lab.db.vendas 
            VERSION AS OF {snapshots[1]['snapshot_id']}
            ORDER BY id
        """).show()
        
        # Time travel por timestamp
        if len(snapshots) >= 1:
            older_timestamp = snapshots[-1]['made_current_at']
            print(f"\n🕒 Dados em timestamp específico ({older_timestamp}):")
            spark.sql(f"""
                SELECT * FROM iceberg.lab.db.vendas 
                TIMESTAMP AS OF '{older_timestamp}'
                ORDER BY id
            """).show()
    else:
        print("⚠️ Poucos snapshots disponíveis para demonstrar time travel")
        
        # Vamos criar mais mudanças para ter mais snapshots
        print("📝 Criando mais alterações para demonstrar time travel...")
        
        # Adicionar mais dados
        spark.sql("""
            INSERT INTO iceberg.lab.db.vendas VALUES 
                (8, 3000.00, 2024),
                (9, 1750.00, 2024)
        """)
        
        # Fazer uma atualização
        spark.sql("""
            UPDATE iceberg.lab.db.vendas 
            SET valor = valor * 1.1 
            WHERE ano = 2024
        """)
        
        print("✅ Novas alterações feitas!")
        
        # Mostrar histórico atualizado
        print("\n📋 Histórico atualizado:")
        spark.sql("DESCRIBE HISTORY iceberg.lab.db.vendas").show()
        
except Exception as e:
    print(f"⚠️ Erro ao demonstrar time travel: {e}")
    print("💡 Time travel requer snapshots múltiplos da tabela")

## 19. Exercício 18: Exportar tabela Iceberg para CSV

In [ ]:
# Exercício 18: Exportar tabela Iceberg para CSV
print("📤 Exportando tabela Iceberg para CSV no HDFS...")

# Ler dados da tabela Iceberg vendas
df_vendas_export = spark.sql("SELECT * FROM iceberg.lab.db.vendas")

print("📊 Dados a serem exportados:")
df_vendas_export.show()

# Caminho de destino no HDFS
export_path = "hdfs://namenode:9000/export/vendas.csv"

# Exportar para CSV
df_vendas_export.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(export_path)

print(f"✅ Dados exportados com sucesso para: {export_path}")

# Verificar se o arquivo foi criado
try:
    import subprocess
    result = subprocess.run(['hdfs', 'dfs', '-ls', '/export/'], 
                          capture_output=True, text=True)
    print("\n📁 Arquivos no diretório /export/:")
    print(result.stdout)
except Exception as e:
    print(f"⚠️ Não foi possível listar arquivos: {e}")

# Verificar conteúdo do arquivo exportado
print("\n🔍 Verificando dados exportados:")
df_verificacao = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(export_path)

df_verificacao.show()
print(f"📈 Total de registros exportados: {df_verificacao.count()}")

# Estatísticas da exportação
print("\n📊 Estatísticas da exportação:")
spark.sql("""
    SELECT 
        'Original' as fonte,
        COUNT(*) as total_registros,
        SUM(valor) as soma_valores,
        AVG(valor) as valor_medio
    FROM iceberg.lab.db.vendas
""").show()

print("✅ Exercício 18 concluído: Tabela Iceberg exportada para CSV com sucesso!")

## 20. Exercício 19: Configuração para Dashboard no Superset

In [ ]:
# Exercício 19: Preparação para Dashboard no Superset
print("📊 Preparando dados para Dashboard no Superset...")

# Criar uma view agregada para facilitar visualizações
print("📈 Criando view agregada para dashboards...")

create_view_sql = """
CREATE OR REPLACE VIEW iceberg.lab.db.vendas_dashboard AS
SELECT 
    ano,
    COUNT(*) as total_vendas,
    SUM(valor) as receita_total,
    AVG(valor) as ticket_medio,
    MIN(valor) as menor_venda,
    MAX(valor) as maior_venda
FROM iceberg.lab.db.vendas
GROUP BY ano
"""

spark.sql(create_view_sql)
print("✅ View 'vendas_dashboard' criada!")

# Verificar dados da view
print("\n📊 Dados da view para dashboard:")
spark.sql("SELECT * FROM iceberg.lab.db.vendas_dashboard ORDER BY ano").show()

# Criar dados de exemplo adicionais para visualizações mais ricas
print("\n📝 Criando tabela de vendas detalhadas para dashboards...")

vendas_detalhadas_sql = """
CREATE OR REPLACE TABLE iceberg.lab.db.vendas_detalhadas
USING ICEBERG AS
SELECT 
    id,
    valor,
    ano,
    CASE 
        WHEN valor < 1000 THEN 'Baixo'
        WHEN valor < 2000 THEN 'Médio' 
        ELSE 'Alto'
    END as categoria_valor,
    CASE 
        WHEN id % 4 = 0 THEN 'Q1'
        WHEN id % 4 = 1 THEN 'Q2'
        WHEN id % 4 = 2 THEN 'Q3'
        ELSE 'Q4'
    END as trimestre,
    valor * 0.1 as comissao
FROM iceberg.lab.db.vendas
"""

spark.sql(vendas_detalhadas_sql)
print("✅ Tabela 'vendas_detalhadas' criada!")

print("\n📊 Preview dos dados detalhados:")
spark.sql("SELECT * FROM iceberg.lab.db.vendas_detalhadas ORDER BY ano, id").show()

print("""
📋 INSTRUÇÕES PARA SUPERSET:

1. Conectar ao Trino:
   - Host: trino (ou IP do container Trino)
   - Porta: 8080
   - Usuário: admin
   - Catálogo: iceberg
   - Schema: lab.db

2. URL de conexão no Superset:
   trino://admin@trino:8080/iceberg/lab.db

3. Tabelas disponíveis para visualização:
   - iceberg.lab.db.vendas (dados brutos)
   - iceberg.lab.db.vendas_dashboard (dados agregados)
   - iceberg.lab.db.vendas_detalhadas (dados com categorias)
   - iceberg.lab.db.pessoas (dados de pessoas)

4. Exemplos de visualizações:
   - Gráfico de barras: receita por ano
   - Gráfico de pizza: vendas por categoria
   - Tabela: métricas de vendas por trimestre
""")

## 21. Exercício 20: Consultas via Trino

In [ ]:
# Exercício 20: Simulação de consultas Trino
print("🔍 Simulando consultas que seriam executadas via Trino...")

# Estas são as consultas que você executaria no CLI do Trino ou Superset
trino_queries = [
    "SELECT * FROM iceberg.lab.db.pessoas;",
    "SELECT * FROM iceberg.lab.db.vendas ORDER BY ano, valor DESC;",
    "SELECT ano, SUM(valor) as total FROM iceberg.lab.db.vendas GROUP BY ano;",
    "SELECT categoria, AVG(preco) as preco_medio FROM iceberg.lab.db.produtos GROUP BY categoria;"
]

print("📋 Consultas Trino equivalentes:")
for i, query in enumerate(trino_queries, 1):
    print(f"{i}. {query}")

print("\n" + "="*80)
print("🚀 Executando consultas via Spark (equivalente ao Trino):")

# Query 1: Pessoas
print("\n1️⃣ Consulta: Tabela pessoas")
try:
    spark.sql("SELECT * FROM iceberg.lab.db.pessoas").show()
except Exception as e:
    print(f"⚠️ Erro: {e}")

# Query 2: Vendas ordenadas
print("\n2️⃣ Consulta: Vendas ordenadas por ano e valor")
try:
    spark.sql("SELECT * FROM iceberg.lab.db.vendas ORDER BY ano, valor DESC").show()
except Exception as e:
    print(f"⚠️ Erro: {e}")

# Query 3: Total por ano
print("\n3️⃣ Consulta: Total de vendas por ano")
try:
    spark.sql("SELECT ano, SUM(valor) as total FROM iceberg.lab.db.vendas GROUP BY ano ORDER BY ano").show()
except Exception as e:
    print(f"⚠️ Erro: {e}")

# Query 4: Preço médio por categoria
print("\n4️⃣ Consulta: Preço médio por categoria de produtos")
try:
    spark.sql("SELECT categoria, AVG(preco) as preco_medio FROM iceberg.lab.db.produtos GROUP BY categoria").show()
except Exception as e:
    print(f"⚠️ Erro: {e}")

print("""
💡 COMANDOS TRINO CLI:

Para conectar ao Trino CLI (fora do notebook):
$ trino --server localhost:8080 --catalog iceberg --schema lab.db

Comandos úteis no Trino:
- SHOW CATALOGS;
- SHOW SCHEMAS FROM iceberg;
- SHOW TABLES FROM iceberg.lab.db;
- DESCRIBE iceberg.lab.db.vendas;

✅ Exercício 20 concluído: Consultas Trino simuladas com sucesso!
""")

## 🎯 Resumo Final Completo

In [ ]:
# 🎯 RESUMO FINAL - Verificação de todos os 20 exercícios
print("🎯 RESUMO FINAL DOS EXERCÍCIOS DE BIG DATA")
print("="*70)

# Verificar todas as tabelas criadas
print("📋 Verificando tabelas Iceberg criadas:")
try:
    spark.sql("SHOW TABLES IN iceberg.lab.db").show()
except Exception as e:
    print(f"⚠️ Erro ao listar tabelas: {e}")

print("\n📊 Resumo dos dados:")

# Verificação das tabelas principais
tabelas_para_verificar = [
    ("pessoas", "iceberg.lab.db.pessoas"),
    ("vendas", "iceberg.lab.db.vendas"),
    ("produtos", "iceberg.lab.db.produtos"),
    ("vendas_dashboard", "iceberg.lab.db.vendas_dashboard"),
    ("vendas_detalhadas", "iceberg.lab.db.vendas_detalhadas")
]

for nome, tabela in tabelas_para_verificar:
    try:
        count = spark.sql(f"SELECT COUNT(*) FROM {tabela}").collect()[0][0]
        print(f"✅ {nome}: {count} registros")
    except Exception as e:
        print(f"⚠️ {nome}: Erro ao verificar - {e}")

print("\n✅ LISTA COMPLETA DOS 20 EXERCÍCIOS:")
exercicios = [
    "✓  1. DataFrame simples criado",
    "✓  2. DataFrame salvo no HDFS como CSV", 
    "✓  3. CSV lido do HDFS",
    "✓  4. Namespace Iceberg criado",
    "✓  5. Tabela Iceberg criada",
    "✓  6. Dados inseridos na tabela Iceberg",
    "✓  7. Tabela Iceberg consultada",
    "✓  8. Registros contados",
    "✓  9. Registro atualizado",
    "✓ 10. Registro deletado",
    "✓ 11. Tabela particionada criada",
    "✓ 12. Dados particionados inseridos",
    "✓ 13. Partição específica consultada",
    "✓ 14. Metadados da tabela visualizados",
    "✓ 15. Tabela criada a partir de DataFrame",
    "✓ 16. Conversão para Iceberg demonstrada",
    "✓ 17. Time Travel implementado",
    "✓ 18. Dados exportados para CSV",
    "✓ 19. Preparação para Superset realizada",
    "✓ 20. Consultas Trino simuladas"
]

for exercicio in exercicios:
    print(exercicio)

print(f"\n🎉 PARABÉNS! Todos os {len(exercicios)} exercícios foram completados!")

print("\n💡 TECNOLOGIAS UTILIZADAS:")
tecnologias = [
    "🐍 Python + PySpark",
    "🧊 Apache Iceberg (formato de tabela)",
    "🗄️ Hive Metastore (metadados)",
    "🐘 HDFS (armazenamento distribuído)",
    "🔍 Trino (engine de consulta)",
    "📊 Apache Superset (dashboards)",
    "⚡ Spark SQL (processamento)"
]

for tech in tecnologias:
    print(f"   {tech}")

print("\n🚀 PRÓXIMOS PASSOS SUGERIDOS:")
next_steps = [
    "1. Execute este notebook no ambiente lab",
    "2. Conecte o Superset ao Trino para criar dashboards",
    "3. Explore funcionalidades avançadas do Iceberg",
    "4. Pratique com datasets maiores e mais complexos",
    "5. Implemente pipelines de dados end-to-end",
    "6. Estude otimizações de performance com particionamento",
    "7. Experimente com schema evolution no Iceberg"
]

for step in next_steps:
    print(f"   {step}")

print("\n" + "="*70)
print("🎓 NOTEBOOK DE EXERCÍCIOS COMPLETO!")
print("📚 Bom estudo com Big Data, Spark e Iceberg!")
print("="*70)